# **Decision Trees: Introduction**

The purpose of this notebook is to nuild from scrach Decision Tree classifer (Classification tree) using CART algorithm is learning algorithm. Decision trees are part of the foundation for Machine Learning. Although they are quite simple, they are very flexible and pop up in a very wide variety of situations.

A Classification Tree makes a statement and then makes a decision on whether or not the statement is True or False. 

**Import packages**

In [ ]:
import pandas as pd
import numpy as np
import copy

**Definine variables**

In [ ]:
col_popcorn = "Love Popcorn"
col_soda = "Love Soda"
col_age = "Age"
col_target = "Love Ice"

In [ ]:
cont_variables = {col_popcorn : False,
                  col_soda : False,
                  col_age : True}

**Create dataframe**

In [ ]:
details = {col_popcorn : ["Yes", "Yes", "No", "No", "Yes", "Yes","No"],
           col_soda : ["Yes", "No", "Yes", "Yes", "Yes", "No","No"],
           col_age : [7, 12, 18, 35, 38, 50, 83],
           col_target : ["No", "No", "Yes", "Yes", "Yes", "No","No"]}

df_ice = pd.DataFrame(details)

In [ ]:
df_ice

,Love Popcorn,Love Soda,Age,Love Ice
0,Yes,Yes,7,No
1,Yes,No,12,No
2,No,Yes,18,Yes
3,No,Yes,35,Yes
4,Yes,Yes,38,Yes
5,Yes,No,50,No
6,No,No,83,No


***Gini Impurity*** is a measurement used to build Decision Trees to determine how the features of a dataset should split nodes to form the tree. More precisely, Gini impurity ranges values from 0 to 0.5., which indicates the likelihood of new, random data being misclassified if it were given a random class label according to the class distribution in the dataset.

**Gini Impirity : the math**

Let $X_k(X_1, X_2, ..., X_{k-1}, X_k)$ denote the features in our dataset $D$ and $Y(y_1, y_2, ..., y_{l-1}, y_l)$ contains samples from $l$ classes. The probability of a datapoint belonging to class $i$ at a given node can be denoted as $p_i$. The Gini Impurity of %D% can be defined as:
$$Gini = 1 - \sum_{i=1}^lp_{i}^2 $$

In [ ]:
def compute_gini(label_count):
  """
  Compute Gini metric which measure the degree of randomness
  Params:
    label_count (dict) : Consist of a column feature values and their count
  """
  prob_sum = 0
  total_count = sum(label_count.values())
  for v in label_count.values():
    p_i = (v/total_count)**2
    prob_sum += p_i
  return 1 - prob_sum

A Gini Impurity of 0 means there’s no impurity, so the data in our node is completely pure. Completely pure means the elements in the node belong to only one category.

<img src="https://cdn-images-1.medium.com/max/730/1*3SMk52FU2a3TrDysEu7dkQ.png" align="center" alt="sigmoid function" width="500"/><br>

In [ ]:
def compute_gini_impurity(df, feature, target, is_continuous=False):
  """
  Compute Gini Impurity for a given feature
  Params :
    df (DataFrame) : Input DataFrame
    feature (str) : Feature variable
    Target (str) : Target variable
    is_continuous (boolean) : Default is False. Says whether a not the feature variable to use in tree node is continuous
  Returns:
    gini (float) : Computed gini impurity 
  """
  if is_continuous:
    gini, split_value = handle_continuous_variable(df, feature, target)
    return gini, split_value
  else:
    return handle_categorical_variable(df, feature, target)

def handle_categorical_variable(df, feature, target):
  """
  Compute Gini Impurity for a categorical variable
  Params :
    df (DataFrame) : Input DataFrame
    feature (str) : Feature variable
    Target (str) : Target variable
  Returns:
    gini (float) : Computed gini impurity
  """
  gini = 0
  label_dict = df[feature].value_counts().to_dict()
  total_count = sum(label_dict.values())
  for k,v in label_dict.items():
    df_subset = df[df[feature]==k]
    k_dict = df_subset[target].value_counts().to_dict()
    n_k = sum(k_dict.values())
    gini += (n_k/total_count)*compute_gini(k_dict)
  return gini

def handle_continuous_variable(df, feature, target):
  """
  Compute Gini Impurity for a continuous variable
  Params :
    df (DataFrame) : Input DataFrame
    feature (str) : Feature variable
    Target (str) : Target variable
  Returns:
    gini (float) : Computed gini impurity
  """
  gini_dict = {}
  df = df.sort_values([feature])
  total_count = len(df)
  values = df[feature].to_list()
  avg_values = [(values[i]+values[i+1])/2 for i in range(len(values)-1)]
  for v in avg_values:
    df_subset_left = df[df[feature]<=v][target]
    label_count_left = df_subset_left.value_counts().to_dict()
    n_k_left = sum(label_count_left.values())

    df_subset_right = df[df[feature]>v][target]
    label_count_right = df_subset_right.value_counts().to_dict()
    n_k_right = sum(label_count_right.values())
    
    gini = (n_k_left/total_count)*compute_gini(label_count_left) + (n_k_right/total_count)*compute_gini(label_count_right)
    gini_dict[v] = gini

  candidate_value = min(gini_dict, key=gini_dict.get)  
  return gini_dict[candidate_value], candidate_value

In [ ]:
class DecisionTreeClassifier():
  """
  Class to build a decision tree classifier
  Params:
    df (DataFrame) : Input DataFrame
    target (str) : The label/target column name
    attributes_dict (dict) : Dictionary containing features as keys and a boolean as value indicating whether or not the future is a continuous variable
    max_depth (int) : Maximum depth of the tree. Parameter to avoid overfitting
  """
  def __init__(self, df, target, attributes_dict, max_depth, min_samples_split):
    self.df = df
    self.target = target
    self.attributes_dict = attributes_dict
    self.max_depth = max_depth
    self.min_samples_split = min_samples_split
    self.depth = 0

  def build_tree(self, dataframe, tree={}):
    """
    Build the decision tree. The function returns a dictionary containing all nodes and leaves with the predicted labls
    """
    if is_leaf(dataframe, self.target):
      return f"{self.target} prediction : {next(iter(set(list(dataframe[self.target]))))}"
    elif stopping_criteria(dataframe, self.depth, self.max_depth, self.min_samples_split):
      label = dataframe[self.target].value_counts().to_dict()
      return f"{self.target} prediction : {max(label, key=label.get)}" 
    else:
      self.depth = self.depth + 1
      #Find best candidate for  Node
      colname, gini, split_value = find_best_split(dataframe, self.attributes_dict, self.target)
      #print(f"Best column split {colname}")
      question = node_feature(self.attributes_dict, colname, default=split_value)
      df_left, df_right = get_right_left_branches(dataframe, self.attributes_dict, colname, self.target, split_value)
      self.attributes_dict.pop(colname, None)
      print(f"Attribute dict after {colname} removal is : {self.attributes_dict}")
      tree = {"Split Question" : question,
              "Feature" : colname,
              "Gini Impurity" : gini}
      tree["left"] = self.build_tree(df_left)
      tree["right"] = self.build_tree(df_right)
      return tree

def is_leaf(df, target):
  """
  Check whether or not we have a Leaf Node
  """
  if df is not None:
    label = df[target].value_counts().to_dict()
    return len(label)==1
  else:
    return None

def stopping_criteria(df, depth, max_depth, min_samples_split):
  """
  Evaluate the stopping criteria. 
  If, for example, there are fewer samples than the minimum required samples per split remaining
  """
  if df is not None:
    n_sample = len(df)
    if n_sample < min_samples_split or depth > max_depth:
      return True
    return False
  else :
    return None

def get_right_left_branches(df, attr_dict, attr_name, target, default=None):
  """
  Partition data into right and left tree branches 
  Params:
    df (DataFrame) : Input dataframe
    attr_dict (dict) : Attribute type dictionary defined at the beginning of this notebook
    default (float) : Default is None. This parameter is for continuous feature variable value to be used to split node
  Returns
    df_left (DataFrame) : Input data filtered to be used at tree left branch
    df_right (DataFrame) : Input data filtered to be used at tree right branch
  """
  if attr_dict[attr_name]:
    df_left = df[df[attr_name]<=default]
    df_right = df[df[attr_name]>default]
    return df_left, df_right
  else:
    label = list(df[attr_name].value_counts().to_dict().keys())
    if len(label)>2:
      val = best_cat_value_split(df, attr_name, label, target)
      print(f"Best split value for attribute {attr_name} is : {val}")
      df_left = df[df[attr_name]==val]
      df_right = df[df[attr_name]!=val]
      return df_left, df_right
    elif len(label)==2:
      print(f"Best split value is ==> {attr_name}")
      df_left = df[df[attr_name]==label[0]]
      df_right = df[df[attr_name]==label[1]]
      return df_left, df_right
    else:
      return df[df[attr_name]==label[0]], None

def best_cat_value_split(df, colname, label, target):
  """
  Get the feature value that has the lowest gini impurity if the column contains more than 2 values
  """
  gini_dict = {}
  for val in label:
    df_subset = df[df[colname]==val]
    k_dict = df_subset[target].value_counts().to_dict()
    gini_dict[val] = compute_gini(k_dict)
  return min(gini_dict, key=gini_dict.get)

def find_best_split(df, attributes, target):
  """
  Find the best attribute on which to build the tree
  """
  gini_dict = {}
  continuous_dict = {}
  con_split_value = 0
  for attr, value in attributes.items():
    if value:
      gini, feature_value = compute_gini_impurity(df, attr, target, is_continuous=value)
      gini_dict[attr] = gini
      continuous_dict[attr] = feature_value
    else:
      gini = compute_gini_impurity(df, attr, target)
      gini_dict[attr] = gini

  best_candidate = min(gini_dict, key=gini_dict.get)
  if attributes[best_candidate]:
    con_split_value = continuous_dict[best_candidate]
  return best_candidate, gini_dict[best_candidate], con_split_value

def node_feature(attributes, attr_name, default=None, condition=None):
  """
  This function return a question to ask. This a question used to split the tree
  """
  if not attributes[attr_name]:
    return f"Node feature variable is : {attr_name}"
  else:
    condition = "<="
    return f"Is {attr_name} {condition} {default}"

**Test code**

In [ ]:
tree = DecisionTreeClassifier(df_ice, col_target, cont_variables, 5, 2)

In [ ]:
tree.build_tree(df_ice)

Best split value is ==> Love Soda
Attribute dict after Love Soda removal is : {'Love Popcorn': False, 'Age': True}
Attribute dict after Age removal is : {'Love Popcorn': False}


{'Feature': 'Love Soda',
 'Gini Impurity': 0.21428571428571427,
 'Split Question': 'Node feature variable is : Love Soda',
 'left': {'Feature': 'Age',
  'Gini Impurity': 0.0,
  'Split Question': 'Is Age <= 12.5',
  'left': 'Love Ice prediction : No',
  'right': 'Love Ice prediction : Yes'},
 'right': 'Love Ice prediction : No'}